In [8]:
import pandas as pd
from glob import glob

In [9]:
file_lists = glob('./*.xlsx')
file_lists

['./대한민국행복지도_안전.xlsx',
 './대한민국행복지도_건강.xlsx',
 './대한민국행복지도_교육.xlsx',
 './대한민국행복지도_여가.xlsx',
 './대한민국행복지도_삶의만족도.xlsx',
 './대한민국행복지도_관계및사회참여.xlsx',
 './대한민국행복지도_경제.xlsx',
 './대한민국행복지도_환경.xlsx']

In [10]:
df_lists = [pd.read_excel(file)  for file in file_lists]

In [13]:
for df in df_lists:
  print(df.shape)

(229, 9)
(229, 9)
(229, 8)
(229, 9)
(229, 4)
(229, 9)
(229, 9)
(229, 9)


In [20]:
# 시도 데이터 추출
happy_merge = pd.DataFrame(df_lists[0]['시도'].unique(),columns=['시도'])
happy_merge

,시도
0,서울특별시
1,부산광역시
2,대구광역시
3,인천광역시
4,광주광역시
5,대전광역시
6,울산광역시
7,세종특별자치시
8,경기도
9,강원도


In [21]:
# 시도별 삶의 만족도의 평균
df_lists[4]

,No,시도,구군,삶의 만족도
0,1,서울특별시,종로구,0.4437
1,2,서울특별시,중구,0.4976
2,3,서울특별시,용산구,0.6161
3,4,서울특별시,성동구,0.4729
4,5,서울특별시,광진구,0.4041
...,...,...,...,...
224,225,경상남도,함양군,0.9565
225,226,경상남도,거창군,0.6163
226,227,경상남도,합천군,0.8057
227,228,제주특별자치도,제주시,0.7113


In [ ]:
grouped =df_lists[4].groupby(['시도'])
for key,group in grouped:
  print('key : ',key)
  print('group ', group)

In [31]:
happy_merge = grouped[['삶의 만족도']].mean()
happy_merge = happy_merge.reset_index()
happy_merge = happy_merge.sort_values(by='삶의 만족도', ascending=False)
happy_merge.head()

,시도,삶의 만족도
9,세종특별자치시,0.907700
14,제주특별자치도,0.711300
0,강원도,0.619506
13,전라북도,0.608193
15,충청남도,0.553100


In [ ]:
# 시도별 삶의 만족도,건강,안전,환경,경제,교육,관계및사회참여,여가 들의 평균을구해서 합친다

In [53]:
# 규칙찾기
file_lists[0].find('_')
file_lists[0].find('.',1)
file_lists[0][10+1:13]
# 파일리스트에 적용해서 단어 분리하기
titles = [file[file.find('_')+1 : file.find('.',1)  ] for file in file_lists]
titles

['안전', '건강', '교육', '여가', '삶의만족도', '관계및사회참여', '경제', '환경']

In [70]:
mege_df = []
for idx, df in enumerate(df_lists):
  try:
    temp = pd.DataFrame(df.groupby(['시도'])['평균'].mean()).reset_index().rename(columns={"평균": titles[idx]})
    mege_df.append(temp)
  except:
    print(idx)

4


In [ ]:
pd.DataFrame(df_lists[0].groupby(['시도'])['평균'].mean()).reset_index().rename(columns={"평균": "안전"})

In [ ]:
pd.merge(mege_df[0],mege_df[1], on='시도')

In [72]:
from functools import reduce

In [74]:
merged = reduce( lambda left,right:pd.merge(left,right, on='시도') , mege_df)
happy = pd.merge( happy_merge, merged, on='시도')
happy.head()

,시도,삶의 만족도,안전,건강,교육,여가,관계및사회참여,경제,환경
0,세종특별자치시,0.907700,0.157800,0.232000,0.587000,0.447800,0.673700,0.511900,0.652400
1,제주특별자치도,0.711300,0.446100,0.253700,0.646800,0.694800,0.610700,0.425000,0.684200
2,강원도,0.619506,0.548000,0.329506,0.488533,0.632750,0.563122,0.386861,0.639839
3,전라북도,0.608193,0.410914,0.421300,0.446936,0.667900,0.487464,0.196543,0.670586
4,충청남도,0.553100,0.481313,0.347433,0.648047,0.625347,0.448720,0.402893,0.558173


In [75]:
# 결측치 확인
happy.isnull().sum()

시도         0
삶의 만족도     0
안전         0
건강         0
교육         0
여가         0
관계및사회참여    0
경제         0
환경         0
dtype: int64

In [76]:
happy.describe()

,삶의 만족도,안전,건강,교육,여가,관계및사회참여,경제,환경
count,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000
mean,0.518189,0.420882,0.409401,0.578653,0.464910,0.497632,0.407635,0.579406
std,0.138257,0.123557,0.127636,0.125408,0.176206,0.114394,0.111265,0.097299
min,0.362081,0.157800,0.232000,0.399412,0.153587,0.294719,0.196543,0.393440
25%,0.411480,0.358429,0.329506,0.488533,0.347960,0.402183,0.386861,0.515020
50%,0.490972,0.446100,0.401064,0.587000,0.454980,0.487464,0.410820,0.582380
75%,0.553100,0.484835,0.482325,0.648047,0.632750,0.601532,0.468926,0.652400
max,0.907700,0.573155,0.663580,0.840780,0.702891,0.676440,0.642580,0.735600


행복지수 요소별 상관관계 확인

In [77]:
happy.corr()

<ipython-input-77-b7797727d096>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  happy.corr()


,삶의 만족도,안전,건강,교육,여가,관계및사회참여,경제,환경
삶의 만족도,1.000000,-0.191709,-0.570466,-0.113522,0.497880,0.519122,-0.057815,0.572262
안전,-0.191709,1.000000,-0.210138,-0.519065,0.382730,-0.444827,-0.142781,-0.124581
건강,-0.570466,-0.210138,1.000000,0.242186,-0.405151,-0.175048,0.071809,-0.357841
교육,-0.113522,-0.519065,0.242186,1.000000,-0.070109,0.462585,0.451395,-0.146646
여가,0.497880,0.382730,-0.405151,-0.070109,1.000000,0.216653,-0.367461,0.494416
관계및사회참여,0.519122,-0.444827,-0.175048,0.462585,0.216653,1.000000,0.298562,0.169838
경제,-0.057815,-0.142781,0.071809,0.451395,-0.367461,0.298562,1.000000,-0.716187
환경,0.572262,-0.124581,-0.357841,-0.146646,0.494416,0.169838,-0.716187,1.000000


한글폰트

In [78]:
!sudo apt-get install -y fonts-nanum

!sudo fc-cache -fv

!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 3s (3,768 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120511 files and dire

In [1]:
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'NanumBarunGothic' # 나눔바른고딕 적용하기